# Model design

Two qubits that evolve with the same Lindbladian (or different ones, it can come in a jnp.array where the first axis is the number of components)

There should be a way of check errors in jax, but I do not know. Probably try catch before calling something with jit

I guess that the main loop can be done with jit, but before we could check that everything is correct. 

For now let's focus on writing things modular.

1. Write everything in row based vectorization to make it compatible with flat and reshape
2. Work as much as we can with vectorized version
3. 

In [1]:
import jax
import jax.numpy as jnp
from qdots_qll.models.game import *

import qutip as qt

In [14]:
seed = 3
rho_ex = qt.rand_dm_ginibre(2, seed=seed)

In [15]:
print(rho_ex)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.9193551 +0.j         0.06815597+0.09741774j]
 [0.06815597-0.09741774j 0.0806449 +0.j        ]]


In [16]:
qt.tensor([qt.rand_dm_ginibre(2, seed=seed), qt.rand_dm_ginibre(3, seed=seed)])

Quantum object: dims = [[2, 3], [2, 3]], shape = (6, 6), type = oper, isherm = True
Qobj data =
[[ 0.38788866+0.j          0.02818244+0.10552207j  0.20733463-0.16244769j
   0.02875595+0.04110189j -0.00909216+0.01080913j  0.03258412+0.00992684j]
 [ 0.02818244-0.10552207j  0.17207868+0.j         -0.08802428-0.1626816j
   0.01327074-0.00483653j  0.01275697+0.01823399j  0.01071261-0.02138765j]
 [ 0.20733463+0.16244769j -0.08802428+0.1626816j   0.35938776+0.j
  -0.00184281+0.03401281j -0.02376389+0.002733j    0.02664305+0.03808185j]
 [ 0.02875595-0.04110189j  0.01327074+0.00483653j -0.00184281-0.03401281j
   0.0340252 +0.j          0.00247214+0.00925629j  0.01818718-0.01424975j]
 [-0.00909216-0.01080913j  0.01275697-0.01823399j -0.02376389-0.002733j
   0.00247214-0.00925629j  0.01509457+0.j         -0.0077214 -0.01427027j]
 [ 0.03258412-0.00992684j  0.01071261+0.02138765j  0.02664305-0.03808185j
   0.01818718+0.01424975j -0.0077214 +0.01427027j  0.03152513+0.j        ]]

In [17]:
qt.tensor([rho_ex, rho_ex])

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.8452138 +0.j          0.06265954+0.0895615j   0.06265954+0.0895615j
  -0.00484498+0.0132792j ]
 [ 0.06265954-0.0895615j   0.0741413 +0.j          0.01413545+0.j
   0.00549643+0.00785624j]
 [ 0.06265954-0.0895615j   0.01413545+0.j          0.0741413 +0.j
   0.00549643+0.00785624j]
 [-0.00484498-0.0132792j   0.00549643-0.00785624j  0.00549643-0.00785624j
   0.0065036 +0.j        ]]

In [2]:
from qbism import sic_povm

In [3]:
sic_povm(4)

[Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
 Qobj data =
 [[ 0.05897908+0.j         -0.04847955+0.00434903j  0.        +0.02442994j
    0.07290949+0.05463006j]
  [-0.04847955-0.00434903j  0.04016985+0.j          0.00180143-0.02008089j
   -0.0559017 -0.05028098j]
  [ 0.        -0.02442994j  0.00180143+0.02008089j  0.01011921+0.j
    0.02262851-0.0302001j ]
  [ 0.07290949-0.05463006j -0.0559017 +0.05028098j  0.02262851+0.0302001j
    0.14073186+0.j        ]],
 Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
 Qobj data =
 [[ 0.05897908+0.j          0.00434903+0.04847955j  0.        -0.02442994j
   -0.05463006+0.07290949j]
  [ 0.00434903-0.04847955j  0.04016985+0.j         -0.02008089-0.00180143j
    0.0559017 +0.05028098j]
  [ 0.        +0.02442994j -0.02008089+0.00180143j  0.01011921+0.j
   -0.0302001 -0.02262851j]
  [-0.05463006-0.07290949j  0.0559017 -0.05028098j -0.0302001 +0.02262851j
    0.14073186+0.j        ]],
 Quan

In [4]:
import jax.typing

In [5]:
dir(jax.typing)

['ArrayLike',
 'DTypeLike',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [6]:
import equinox as eqx

In [97]:
from jax import Array

# from jax.typing import ArrayLike
from jaxtyping import Array, Float, Complex


class base_class_dimension(eqx.Module):
    d: int
    matrix_d: Float[Array, "d d"]
    positive_eps: Float

    def __init__(
        self,
        dimension: int,
    ) -> None:
        self.d = dimension
        self.matrix_d = jnp.ones([self.d, self.d])
        self.positive_eps = 1e-8

    @jit
    def vec(self, rho: Complex[Array, "d d"]) -> Complex[Array, "d**2"]:
        return rho.flatten()

    @jit
    def sprepost(
        self, A: Complex[Array, "d1 d2"], B: Complex[Array, "d3 d4"]
    ) -> Float[Array, "d1*d2 d2*d4"]:
        return jnp.kron(A, B.T)

    @jit
    def spre(self, A: Complex[Array, "d d"]) -> Complex[Array, "d**2 d**2"]:
        return self.sprepost(A, jnp.identity(self.matrix_d.shape[0]))

    @jit
    def spost(self, A: Complex[Array, "d d"]) -> Complex[Array, "d**2 d**2"]:
        return self.sprepost(jnp.identity(self.matrix_d.shape[0]), A)

In [96]:
a = base_class_dimension(2)
a.spre(rho_ex.full())

Array([[0.9193551 +0.j        , 0.        +0.j        ,
        0.06815597+0.09741774j, 0.        +0.j        ],
       [0.        +0.j        , 0.9193551 +0.j        ,
        0.        +0.j        , 0.06815597+0.09741774j],
       [0.06815597-0.09741774j, 0.        +0.j        ,
        0.0806449 +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.06815597-0.09741774j,
        0.        +0.j        , 0.0806449 +0.j        ]], dtype=complex64)

In [98]:
class single_qdot(base_class_dimension):
    number_of_parameters: int
    delta: float
    Omega: float
    system_hamiltonian: Complex[Array, "d d"]
    A: Complex[Array, "d d"]
    sic_POVM_arr: jax.Array
    basis_elements: jax.Array
    # true_parameters: jax.Array

    def __init__(self):
        super().__init__(dimension=2)
        self.number_of_parameters = 4
        self.delta = 0.12739334807998307
        self.Omega = 0.5
        self.system_hamiltonian = self.make_system_hamiltonian()
        self.A = jnp.array([[1, 0], [0, 0]])
        self.sic_POVM_arr = get_SIC_POVM_4D()
        self.basis_elements = jnp.identity(4)

    @jit
    def make_system_hamiltonian(self) -> Array:
        return jnp.array([[self.delta, self.Omega / 2], [self.Omega / 2, 0]])

    @jit
    def make_liouvillian(self, particle: Array) -> Array:
        gn, gp, Sn, Sp = particle
        Snot = -self.delta
        gnot = self.positive_eps  # if we write zero derivatives explode
        # because this appears in square roots d/dx (sqrt) prop to 1/sqrt

        U = jnp.linalg.eigh(self.system_hamiltonian)[1]
        Aij = U @ self.A @ dag(U)
        Cp = 0.5 * gp + 1j * Sp
        Cn = 0.5 * gn + 1j * Sn
        Cnot = 0.5 * gnot + 1j * Snot

        Gamma = jnp.array([[Cnot, Cn], [Cp, Cnot]])

        sqrtgamma = jnp.sqrt(jnp.real(Gamma))

        L = jnp.multiply(Aij, sqrtgamma)

        Af = jnp.multiply(Aij, jnp.conjugate(Gamma))
        Hrenorm = -1j / 2 * (Aij @ dag(Af) - Af @ dag(Aij))

        Htotal = U @ self.system_hamiltonian @ dag(U) + Hrenorm
        Liouvillian_ebasis = (
            -1j * (self.spre(Htotal) - self.spost(Htotal))
            + self.sprepost(dag(L), L)
            - 0.5 * (self.spre(L @ dag(L)) + self.spost(L @ dag(L)))
        )
        return (
            self.sprepost(U, dag(U))
            @ Liouvillian_ebasis
            @ dag(self.sprepost(U, dag(U)))
        )

    # @jit
    # def generate_choi(self, particle, t):
    #     L = self.make_liouvillian(particle)

    #     evolved_basis_array = vmap(evolve_basis, (None, None, 0))(
    #         t,
    #         L,
    #         self.basis_elements,
    #     )

    #     return (
    #         construct_choi_state(
    #             self.basis_elements.reshape(-1, 2, 2),
    #             evolved_basis_array,
    #         )
    #         / 2
    #     )

    # @jit
    # def likelihood_particle(self, particle, t):
    #     choi_state = self.generate_choi(particle, t)
    #     return jax.vmap(compute_P_singleop, in_axes=(None, 0))(
    #         choi_state, self.sic_POVM_arr
    #     )

    # @jit
    # def fim(self, particle, t):
    #     """Given a particle and time, computes the Fisher Information Matrix

    #     Args:
    #         particle (_type_): (j)np array of particle
    #         t (_type_): time

    #     Returns:
    #         _type_: _description_
    #     """
    #     jacobian = jax.jacobian(self.likelihood_particle, 0)(particle, t)
    #     lkl = self.likelihood_particle(particle, t)
    #     return jnp.einsum("ij, ik, i -> jk", jacobian, jacobian, 1 / lkl)

    # @jit
    # def generate_data(self, key, true_particle, t):
    #     probabilities = self.likelihood_particle(true_particle, t)
    #     probabilities = probabilities / probabilities.sum()
    #     no_of_outcomes = self.sic_POVM_arr.shape[0]
    #     outcome = jax.random.choice(
    #         key, a=jnp.arange(no_of_outcomes), p=probabilities
    #     )
    #     return outcome

In [99]:
m = single_qdot()

In [101]:
m

single_qdot(
  d=2,
  matrix_d=f32[2,2],
  positive_eps=1e-08,
  number_of_parameters=4,
  delta=0.12739334807998307,
  Omega=0.5,
  system_hamiltonian=f32[2,2],
  A=i32[2,2],
  sic_POVM_arr=c64[16,4,4],
  basis_elements=f32[4,4]
)